# はじめに
LLMを実行するためのサンプルコードです。

### 概要
- 小型で性能の高いMistralを動かします。
- 認証などについても説明します。

# 事前準備
1. Hugging Faceのアカウント作成
2. アクセストークンの作成  
Hugging Faceのアカウントにログインし、「自分のアイコン→Settings→Access Tokens→+ Create new token」からトークンを作成します。  
「Read access to contents of all public gated repos you can access」の権限が必要となります。  
3. モデルの利用条件の同意  
Hugging Faceで使用したいモデルのページにアクセスして、利用条件に同意します。


# 実装
### 1. ライブラリのインポート

In [1]:
import sys
import os

!pip install -U bitsandbytes
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.1 MB/s eta 0:00:00


GPU環境を確認します。  

In [2]:
!nvidia-smi

Sat Oct 19 07:12:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### 2. 認証
Hugging Faceからモデルをダウンロードするために、アカウント認証を行います。  
login()を実行するとトークンを入力するUIが出てくるので、そこにトークンを入力します。  
また、colabのシークレットにトークンを保存しておけば、それを利用することもできます。

In [4]:
from huggingface_hub import login
# login()

# シークレットにトークンを保存しておけば下記で認証可能
from google.colab import userdata
login(token=userdata.get('HF_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### 3. モデルの準備
トークナイザーとモデルを準備します。  
モデルはv0.3を4bit量子化したものを使用します。

In [9]:
tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
)

quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=False,
    quantization_config=quantization_config,
)

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

### 4. 推論
プロンプトをモデルに入力して、得られた出力を表示します。

In [11]:
# 推論時間を計測
%%time

# プロンプトの準備
conversation = [{"role": "user", "content": "What's the weather like in Paris?"}]

# 推論の実行
with torch.no_grad():
    token_ids = tokenizer.apply_chat_template(conversation, add_generation_prompt=True,
            return_dict=True, return_tensors="pt").to(model.device)
    output_ids = model.generate(
        **token_ids,
        pad_token_id=tokenizer.eos_token_id,
        temperature=0.1,
        do_sample=True,
        top_p=0.95,
        top_k=40,
        max_new_tokens=256,
    )
output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output)

What's the weather like in Paris? I don't have real-time capabilities, so I can't provide the current weather in Paris. However, Paris, like much of France, has a temperate oceanic climate, characterized by mild winters and relatively warm summers. Rainfall is relatively evenly distributed throughout the year. For a more accurate and up-to-date forecast, I recommend checking a reliable weather service.
CPU times: user 5.51 s, sys: 4.68 ms, total: 5.51 s
Wall time: 5.51 s


# おわりに
### 今回の結果
Mistralでプロンプトの回答を生成することができました。  
天気に関する質問に答えていないので、ハルシネーションを防ぐための調整もされていることが確認できました。

### 次にやること  
プロンプトの設定方法について様々な方法を試してみたいと思います。

### 参考資料  
- mistralai/Mistral-7B-Instruct-v0.3  
https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3
